# RAGnificent
A Magnificent RAG for the IBM Specialization "Generative AI Engineering with LLMs" final project

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import (
    PyMuPDFLoader,
    UnstructuredMarkdownLoader,
    JSONLoader,
    WebBaseLoader
)

In [ ]:
# Used:
facebook_chat_json_path = Path("documents") / "json" / "facebook_chat.json"
markdown_sample_path = 'documents\markdown\markdown-sample.md'
lora_paper_pdf_path = 'documents\pdf\LoRA_paper.pdf'
langchain_url = 'https://www.ibm.com/topics/langchain'

# Unused:
# mlb_teams_csv_path = 'documents\csv\mlb_teams_2012.csv'
# large_scale_alignment_pdf_path = 'documents\pdf\large_scale_alignment.pdf'
# new_policies_txt_path = 'documents\txt\new_policies.txt'

## Task 1 - Load document using LangChain for different sources

### PDF

In [ ]:
pdf_loader = PyMuPDFLoader(lora_paper_pdf_path)
pdf_data = pdf_loader.load()
# print(pdf_data[0])

### Markdown

In [ ]:
md_loader = UnstructuredMarkdownLoader(markdown_sample_path)
md_data = md_loader.load()
# print(md_data[0])

### JSON

In [ ]:
json_loader = JSONLoader(
    file_path=facebook_chat_json_path,
    jq_schema='.messages[].content',
    text_content=False)

json_data = json_loader.load()
# print(json_data[0])

### Web

In [ ]:
web_loader = WebBaseLoader(langchain_url)
web_data = web_loader.load()
# print(web_data[0])